In [2]:
from skimage.metrics import structural_similarity as ssim
import torch
import numpy as np
import math
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import os
import re
import cv2
from tqdm import tqdm
from ultralytics import YOLO
import contextlib
import io
from PIL import Image
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from premium_gan_generator import Generator
from apply_bb import apply_bounding_box_mask


In [3]:
# Thiết lập device cho PyTorch (GPU nếu có)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# Các tham số
SIZE = 128  # Kích thước ảnh đầu vào
NoofIMGs = 9001
batch_size = 32

# Đường dẫn đến các thư mục ảnh
path_1 = 'C:/Sam C-Gan project/data/without_mask/train'
path_2 = 'C:/Sam C-Gan project/data/with_mask/train'

path_3='C:/Sam C-Gan project/data/without_mask/val'
path_4='C:/Sam C-Gan project/data/with_mask/val'
# Hàm sắp xếp alphanumeric
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(data, key=alphanum_key)

# Đọc danh sách file và lấy đường dẫn ảnh
# for train
files_1 = sorted_alphanumeric([f for f in os.listdir(path_1) if os.path.isfile(os.path.join(path_1, f))])[:NoofIMGs]
files_2 = sorted_alphanumeric([f for f in os.listdir(path_2) if os.path.isfile(os.path.join(path_2, f))])[:NoofIMGs]
without_mask_paths = [os.path.join(path_1, fname).replace("\\", "/") for fname in files_1]
with_mask_paths = [os.path.join(path_2, fname).replace("\\", "/") for fname in files_2]

# for test
files_3 = sorted_alphanumeric([f for f in os.listdir(path_3) if os.path.isfile(os.path.join(path_3, f))])[:999]
files_4 = sorted_alphanumeric([f for f in os.listdir(path_4) if os.path.isfile(os.path.join(path_4, f))])[:999]
without_mask_val_paths = [os.path.join(path_3, fname).replace("\\", "/") for fname in files_3]
with_mask_val_paths = [os.path.join(path_4, fname).replace("\\", "/") for fname in files_4]

# Lớp Dataset tùy chỉnh
class MaskDataset(Dataset):
    def __init__(self, without_mask_paths, with_mask_paths, transform=None):
        self.without_mask_paths = without_mask_paths
        self.with_mask_paths = with_mask_paths
        self.transform = transform

    def __len__(self):
        return len(self.without_mask_paths)

    def __getitem__(self, idx):
        # Đọc ảnh không có khẩu trang và có khẩu trang
        without_mask_img = Image.open(self.without_mask_paths[idx]).convert('RGB')
        with_mask_img = Image.open(self.with_mask_paths[idx]).convert('RGB')

        # Áp dụng biến đổi nếu có
        if self.transform:
            without_mask_img = self.transform(without_mask_img)
            with_mask_img = self.transform(with_mask_img)

        return without_mask_img, with_mask_img

# Các phép biến đổi (resize và chuẩn hóa)
transform = transforms.Compose([
    transforms.Resize((SIZE, SIZE)),   # Resize ảnh về kích thước 64x64
    transforms.ToTensor(),             # Chuyển ảnh thành tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Chuẩn hóa về [-1, 1]
])

# Tạo dataset và dataloader
dataset = MaskDataset(without_mask_paths, with_mask_paths, transform=transform)
valset=MaskDataset(without_mask_val_paths, with_mask_val_paths, transform=transform)

# Tạo DataLoader với shuffle và chia batch
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0,drop_last=True)
valloader=DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=0,drop_last=True)


In [5]:
def calculate_ssim(img1, img2):
    """
    Tính toán SSIM giữa hai ảnh.
    """
    img1 = img1.cpu().numpy().transpose(1, 2, 0)  # Chuyển từ (C, H, W) -> (H, W, C)
    img2 = img2.cpu().numpy().transpose(1, 2, 0)  # Chuyển từ (C, H, W) -> (H, W, C)

    # Đảm bảo dữ liệu nằm trong [0, 1]
    img1 = np.clip(img1, 0, 1)
    img2 = np.clip(img2, 0, 1)

    # Sử dụng cửa sổ nhỏ hơn để tránh lỗi
    return ssim(img1, img2, win_size=5, channel_axis=2, data_range=img2.max() - img2.min())


def calculate_psnr(img1, img2):
    """
    Tính PSNR giữa hai ảnh.
    """
    mse = torch.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = 1.0  # Nếu ảnh đã được chuẩn hóa về [0, 1]
    psnr = 20 * math.log10(max_pixel / math.sqrt(mse.item()))
    return psnr

In [6]:
def evaluate_generator_on_val(generator, valloader, device,model_yolo):
    """
    Đánh giá Generator trên tập validation bằng SSIM và PSNR.
    """
    total_ssim = 0
    total_psnr = 0
    num_images = 0

    with torch.no_grad():
        for batch_idx, (without_mask_batch, with_mask_batch) in enumerate(valloader):
            # Đưa batch ảnh vào GPU (nếu có)
            with_mask_batch = with_mask_batch.to(device)
            without_mask_batch = without_mask_batch.to(device)

            # Tạo ảnh bằng Generator
            images_with_mask = apply_bounding_box_mask(with_mask_batch, model_yolo)
            images_with_mask = images_with_mask.to(device)
            generated_images = generator(images_with_mask.permute(0, 3, 1, 2)).detach()  

            # Duyệt từng cặp ảnh trong batch
            for real_img, gen_img in zip(without_mask_batch, generated_images):
                # Chuẩn hóa ảnh từ [-1, 1] về [0, 1]
                real_img = real_img * 0.5 + 0.5
                gen_img = gen_img * 0.5 + 0.5

                # Tính SSIM
                ssim_value = calculate_ssim(real_img, gen_img)
                total_ssim += ssim_value

                # Tính PSNR
                psnr_value = calculate_psnr(real_img, gen_img)
                total_psnr += psnr_value

                num_images += 1

    # Tính giá trị trung bình
    avg_ssim = total_ssim / num_images
    avg_psnr = total_psnr / num_images

    print(f"Đánh giá trên tập Validation:")
    print(f"  - SSIM trung bình: {avg_ssim:.4f}")
    print(f"  - PSNR trung bình: {avg_psnr:.4f}")

    return avg_ssim, avg_psnr


In [ ]:
# Premium GAN evaluation mertric using SSIM and PSNR
generator = Generator()
generator.load_state_dict(torch.load('C:\\Sam C-Gan project\\checkpoints\\generator_epoch_60.pth'))
generator.to(device)
generator.eval()  # Đặt mô hình ở chế độ đánh giá
model_yolo=YOLO('C:\\Sam C-Gan project\\bounding black box\\mask_detection.pt')
ssim,psnr=evaluate_generator_on_val(generator,valloader,device,model_yolo)
print(f'SSIM={ssim}     PSNR={psnr}')

Đánh giá trên tập Validation:
  - SSIM trung bình: 0.9383
  - PSNR trung bình: 30.6057
SSIM=0.9383131265640259     PSNR=30.605732532567977
